# Trade
- Swing trading/long-term trading
    - Exposed to overnight risk (close price previous day might not equal to open 
    price next day if there are major events happening between market closure and
    market open).
- Assume I already have which day to long, which day to short
- Conduct post-trade analysis
- Refine risk management techniques (Comparing starting on 2023-12-22)
    - Boeing: Main character in the events
        - Stock -18.61%

    - Direct competitors
        - Airbus (EPA: AIR): Boeing's primary competitor in commercial aircraft manufacturing
            - Stock +5.93%
        - Lockhead Martin (LMT): More focused on defense but also compete in aerospace
    - Suppliers
        - General Electric (GE): Supplies engines for Boeing aircraft
            - Have presence in aviation, healthcare, power, renewable energy
            - Doesn't seem to be affected
            - Can also supply engines to other aircraft manufacturers (effect on
            stock price is complicated)
    - Customers
        - Alaska Airlines (ALK): Main airline involved
            - Stock -11.73%
        - American Airlines (UAL - NasdaqGS)
            - Stock -4.91%
        - Delta Air Lines (DAL)
            - -11.73%
        - Southwest Airlines
- Trading timing (NYSE) vs news timing
    - The news was updated on January 18, 2024, at 4:36 AM GMT+8, which translates to January 17, 2024, at 3:36 PM Eastern Time (since GMT+8 is 13 hours ahead of Eastern Time). Since the NYSE closes at 4:00 PM ET, this news would have come out just before the market close.
    - Difference stock exchanges might operate at different timings also
- No training and validation - straight go to validation (backtesting)



In [1]:
import yfinance as yf
from backtesting import Backtest, Strategy
import pandas as pd
import numpy as np

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Similar to download data codes

In [24]:
hist_start = pd.to_datetime('2024-01-04') # Historical start date
hist_end = pd.to_datetime('2024-01-12')
# Define the ticker list
ticker_list = ['BA']

# Fetch the data
dl_data = yf.download(ticker_list, start=hist_start, end=hist_end) # Auto adjust is false

dl_data = pd.DataFrame(dl_data)
data = dl_data.drop(columns=['Close'], axis=1)
data = data.rename(columns={'Adj Close': 'Close'})
display(data.isna().sum(axis=0)) # Axis=0: along the indices, row-wise opertaion
# Gives the sum for rows in a column
data.index = pd.to_datetime(data.index)

# TODO: Add the sentiment scores before the next market-open day
# data['Sentiment'] = np.random.random(len(data)) * 2 - 1
display(len(data))
sentiment = np.array([0, -1, -0.8, 0, 0, 0]) # Put -1 on 01-05 (Before the whole thing Boeing case appeared after market closed on 01-05 to prepare to trade for 01-08)
data['Sentiment'] = sentiment
display(data.tail(20))


[*********************100%%**********************]  1 of 1 completed


Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

6

,Open,High,Low,Close,Volume,Sentiment
Date,,,,,,
2024-01-04,244.580002,248.279999,244.179993,244.940002,5170700,0.0
2024-01-05,245.039993,250.190002,245.039993,249.000000,3846200,-1.0
2024-01-08,228.000000,233.850006,225.789993,229.000000,40730400,-0.8
2024-01-09,225.660004,228.789993,223.199997,225.759995,20687500,0.0
2024-01-10,226.899994,231.610001,226.639999,227.839996,12883700,0.0
2024-01-11,228.070007,228.279999,222.619995,222.660004,11830500,0.0


In [30]:

# Assume your data includes 'Open', 'Close', and 'Sentiment' columns
# data = pd.read_csv('your_stock_data.csv', parse_dates=True, index_col='Date')

class SentimentStrategy(Strategy):
    def init(self):
        super().init()
        # Initialize additional indicators here if needed

    def next(self):
        current_sentiment = self.data.Sentiment[-1]
        
        # Adding a hold condition
        if -0.2 <= current_sentiment <= 0.2:
            return  # Hold if sentiment is within the range

        # Checking trade size (adjust as needed)
        trade_size = 1  # Example trade size, adjust based on your strategy

        # Trading logic based on sentiment score
        if current_sentiment > 0.8:  # Threshold for going long
            if not self.position or self.position.is_short:
                self.position.close()  # Close short position if any
                self.buy(size=trade_size)
                print(f"Buy on: {self.data.index[-1]}, Sentiment: {self.data.Sentiment[-1]}")
        elif current_sentiment < -0.8:  # Threshold for going short
            if not self.position or self.position.is_long:
                self.position.close()  # Close long position if any
                self.sell(size=trade_size)
                print(f"Sell on: {self.data.index[-1]}, Sentiment: {self.data.Sentiment[-1]}")

# Additional considerations
# 1. Slippage: Incorporate slippage into your trades, if your backtesting framework allows.
# 2. Execution Price: Decide whether to execute at the current day's close or the next day's open.
# 3. Risk Management: Implement risk management strategies like stop-loss orders.

# Running the backtest
bt = Backtest(data, SentimentStrategy, cash=10000, commission=.0) # TODO: Adjust commission
stats = bt.run()
display(stats)
bt.plot()

# TODO: Change to use close instead of open, change html location

Sell on: 2024-01-05 00:00:00, Sentiment: -1.0


Start                     2024-01-04 00:00:00
End                       2024-01-11 00:00:00
Duration                      7 days 00:00:00
Exposure Time [%]                   66.666667
Equity Final [$]                  9999.929993
Equity Peak [$]                  10002.240005
Return [%]                            -0.0007
Buy & Hold Return [%]               -9.096105
Return (Ann.) [%]                   -0.029399
Volatility (Ann.) [%]                0.316167
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.023095
Avg. Drawdown [%]                   -0.016547
Max. Drawdown Duration        4 days 00:00:00
Avg. Drawdown Duration        3 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                      -0.030705
Worst Trade [%]                     -0.030705
Avg. Trade [%]                    

/opt/homebrew/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/homebrew/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/homebrew/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/homebrew/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p5660', ...)